In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import japanize_matplotlib
import ydata_profiling as pdp
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from graphviz import Digraph

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot') #グラフのスタイル
plt.rcParams['figure.figsize'] = [12, 9] # グラフサイズ設定
plt.rcParams['font.size'] = 14 #フォントサイズ

In [2]:
#①AirREGIヘルプデスク日次入電数
data1 = pd.read_csv("regi_call_data_transform.csv", 
                    index_col='cdr_date',           
                    parse_dates=True)

#②AirREGIアカウント日次獲得数
data2 = pd.read_csv("regi_acc_get_data_transform.csv",
                    index_col='cdr_date',           
                    parse_dates=True)

#③AirPAYのCM実施フラグ
data3 = pd.read_csv("cm_data.csv", 
                    index_col='cdr_date',          
                    parse_dates=True)

#④GoogleTrendの関連ワード検索数（週次データ）
data4 = pd.read_csv("gt_service_name.csv", 
                    index_col='cdr_date', 
                    parse_dates=True)

#⑤カレンダーデータ
data5 = pd.read_csv("calender_data.csv", 
                    index_col='cdr_date',           
                    parse_dates=True)
                    
#⑥GoogleTrend "モバイル POS"検索数（週次データ）
data6 = pd.read_csv("google_trends.csv", 
                    index_col='cdr_date',          
                    parse_dates=True)

In [3]:
data1.head()

,call_num
cdr_date,
2018-06-01,183
2018-06-02,0
2018-06-03,0
2018-06-04,213
2018-06-05,143


In [4]:
data2.head()

,acc_get_cnt
cdr_date,
2018-05-01,-0.926645
2018-05-02,-0.567495
2018-05-03,-1.050489
2018-05-04,-1.199103
2018-05-05,-1.360101


In [5]:
data3.head()

,cm_flg
cdr_date,
2018-03-01,0
2018-03-02,0
2018-03-03,0
2018-03-04,0
2018-03-05,0


In [6]:
data4.head()

,search_cnt
cdr_date,
2018-03-04,49
2018-03-11,25
2018-03-18,31
2018-03-25,25
2018-04-01,19


In [7]:
data5.head()

,dow,dow_name,woy,wom,doy,financial_year,day_before_holiday_flag,holiday_flag,holiday_name
cdr_date,,,,,,,,,
2018-06-01,5,金,22,1,152,2018,True,False,NaN
2018-06-02,6,土,22,1,153,2018,True,True,NaN
2018-06-03,7,日,22,2,154,2018,False,True,NaN
2018-06-04,1,月,23,2,155,2018,False,False,NaN
2018-06-05,2,火,23,2,156,2018,False,False,NaN


In [8]:
data6.head()

,search_trend
cdr_date,
2017-01-01,24
2017-02-01,0
2017-03-01,33
2017-04-01,30
2017-05-01,0


# データ結合＋特徴量作成

In [9]:
#acc_get_cntが標準化されているため
#StandardScalerを用いて"call_num"カラムを標準化し、データのスケールを合わせる
from sklearn.preprocessing import StandardScaler

df = pd.merge(data1, data2, on='cdr_date', how='inner')
scaler = StandardScaler()
df['call_num'] = scaler.fit_transform(df[['call_num']])
df.head()

,call_num,acc_get_cnt
cdr_date,,
2018-06-01,0.555056,-0.282652
2018-06-02,-0.972049,-1.025720
2018-06-03,-0.972049,-1.075258
2018-06-04,0.805402,-0.220730
2018-06-05,0.221263,-0.431266


In [10]:
#差分系列をつくる
df = df.diff(1).dropna()
df.head()

,call_num,acc_get_cnt
cdr_date,,
2018-06-02,-1.527106,-0.743068
2018-06-03,0.000000,-0.049538
2018-06-04,1.777451,0.854528
2018-06-05,-0.584139,-0.210536
2018-06-06,-0.100138,0.136229


In [11]:
#特徴量の詳細はpdf参照
#目的変数(acc_get_cnt)に関する時系列特徴量

lag1 = df["acc_get_cnt"].shift(1)
lag7 = df["acc_get_cnt"].shift(7)
window7 = lag1.rolling(window=7).mean()
expanding = lag1.expanding().mean()

df = pd.concat([ df,
                 lag1,
                 lag7,
                 window7,
                 expanding],
                 axis = 1)
df.columns = ['call_num', 'acc_get_cnt', 'acc_get_cnt_lag1', 'acc_get_cnt_lag7', 'acc_get_cnt_window7', 'acc_get_cnt_expanding']
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 669 entries, 2018-06-02 to 2020-03-31
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   call_num               669 non-null    float64
 1   acc_get_cnt            669 non-null    float64
 2   acc_get_cnt_lag1       668 non-null    float64
 3   acc_get_cnt_lag7       662 non-null    float64
 4   acc_get_cnt_window7    662 non-null    float64
 5   acc_get_cnt_expanding  668 non-null    float64
dtypes: float64(6)
memory usage: 36.6 KB


In [12]:
#CM実施累積回数データを追加
data3['cm_cumsum'] = data3['cm_flg'].cumsum()
data3.head()

,cm_flg,cm_cumsum
cdr_date,,
2018-03-01,0,0
2018-03-02,0,0
2018-03-03,0,0
2018-03-04,0,0
2018-03-05,0,0


In [13]:
#週次データと日次データの結合
merged_data = pd.merge(data5, data6, on='cdr_date', how = 'left')
merged_data = pd.merge(merged_data, data4, on='cdr_date', how='left')
#週次データを日次データに組み合わせるため、直前の非欠損値で埋める
merged_data['search_trend'] = merged_data['search_trend'].fillna(method='ffill')
merged_data['search_cnt'] = merged_data['search_cnt'].fillna(method='ffill')

#日曜日フラグ
merged_data['isSunday'] = (merged_data['dow'] == 7).astype(int)
merged_data.head(30)

,dow,dow_name,woy,wom,doy,financial_year,day_before_holiday_flag,holiday_flag,holiday_name,search_trend,search_cnt,isSunday
cdr_date,,,,,,,,,,,,
2018-06-01,5,金,22,1,152,2018,True,False,NaN,27.0,NaN,0
2018-06-02,6,土,22,1,153,2018,True,True,NaN,27.0,NaN,0
2018-06-03,7,日,22,2,154,2018,False,True,NaN,27.0,18.0,1
2018-06-04,1,月,23,2,155,2018,False,False,NaN,27.0,18.0,0
2018-06-05,2,火,23,2,156,2018,False,False,NaN,27.0,18.0,0
2018-06-06,3,水,23,2,157,2018,False,False,NaN,27.0,18.0,0
2018-06-07,4,木,23,2,158,2018,False,False,NaN,27.0,18.0,0
2018-06-08,5,金,23,2,159,2018,True,False,NaN,27.0,18.0,0
2018-06-09,6,土,23,2,160,2018,True,True,NaN,27.0,18.0,0


In [14]:
df.reset_index(inplace=True)
df['cdr_date'] = pd.to_datetime(df['cdr_date'])
merged_inner = pd.merge(df, data3, on='cdr_date', how='inner')
dataX = pd.merge(merged_inner, merged_data, on='cdr_date', how='inner')

# "dow_name"カラム、 "holiday_name"カラム、"financial_year"カラムを削除する
dataX.drop(['dow_name', 'holiday_name', 'financial_year'], axis=1, inplace=True)
dataX.sample(30)

,cdr_date,call_num,acc_get_cnt,acc_get_cnt_lag1,acc_get_cnt_lag7,acc_get_cnt_window7,acc_get_cnt_expanding,cm_flg,cm_cumsum,dow,woy,wom,doy,day_before_holiday_flag,holiday_flag,search_trend,search_cnt,isSunday
10,2018-06-12,-0.567449,-0.185767,0.978373,-0.210536,-0.007077,0.001238,0,8,2,24,3,163,False,False,27.0,6.0,0
225,2019-01-13,0.000000,-0.334381,-0.829760,-0.012384,0.024769,-0.002092,0,22,7,2,3,13,True,True,43.0,18.0,1
181,2018-11-30,0.308759,-0.272458,0.037153,-0.470610,0.040692,0.002258,0,22,5,48,5,334,True,False,24.0,44.0,0
565,2019-12-19,0.066759,0.099076,0.173383,0.272458,-0.001769,0.000153,0,160,4,51,3,353,False,False,30.0,65.0,0
159,2018-11-08,0.033379,0.321996,-0.408688,-0.247689,-0.127383,-0.000467,0,22,4,45,2,312,False,False,24.0,25.0,0
4,2018-06-06,-0.100138,0.136229,-0.210536,NaN,NaN,-0.037153,0,8,3,23,2,157,False,False,27.0,18.0,0
505,2019-10-20,0.000000,-0.656377,-0.544917,-0.346765,0.037153,0.000687,0,160,7,42,4,293,False,True,39.0,34.0,1
69,2018-08-10,-0.016690,-0.099076,0.074307,-0.049538,0.014154,-0.002513,0,8,5,32,2,222,True,False,0.0,41.0,0
357,2019-05-25,-1.235036,-1.188909,-0.049538,-0.953604,-0.014154,0.001908,1,80,6,21,4,145,True,True,31.0,23.0,0
609,2020-02-01,-1.043105,-0.668761,-0.321996,-0.507763,0.024769,0.000407,0,160,6,5,1,32,True,True,35.0,37.0,0


In [15]:
dataX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   cdr_date                 669 non-null    datetime64[ns]
 1   call_num                 669 non-null    float64       
 2   acc_get_cnt              669 non-null    float64       
 3   acc_get_cnt_lag1         668 non-null    float64       
 4   acc_get_cnt_lag7         662 non-null    float64       
 5   acc_get_cnt_window7      662 non-null    float64       
 6   acc_get_cnt_expanding    668 non-null    float64       
 7   cm_flg                   669 non-null    int64         
 8   cm_cumsum                669 non-null    int64         
 9   dow                      669 non-null    int64         
 10  woy                      669 non-null    int64         
 11  wom                      669 non-null    int64         
 12  doy                      669 non-nul

# 三角特徴量作成（季節性を捉えるための特徴量）

In [16]:
# 空のデータフレーム生成
exog = pd.DataFrame()
exog.index = dataX.index

# 三角関数特徴量（Fourier terms）の生成関数
def fourier_terms_gen(seasonal,terms_num):
    
    #seasonal:周期
    #terms_num:Fourier termの数（sinとcosのセット数）
    
    for num in range(terms_num):
        num = num + 1
        sin_colname = 'sin'+str(seasonal)+'_'+ str(num)
        cos_colname = 'cos'+str(seasonal)+'_'+ str(num)
        exog[sin_colname] = np.sin(num * 2 * np.pi * exog.index / seasonal)
        exog[cos_colname] = np.cos(num * 2 * np.pi * exog.index / seasonal)
        
# 三角関数特徴量の生成
## １週間周期
fourier_terms_gen(
    seasonal=7,
    terms_num=7)

print(exog) #確認

           sin7_1    cos7_1        sin7_2    cos7_2        sin7_3    cos7_3  \
0    0.000000e+00  1.000000  0.000000e+00  1.000000  0.000000e+00  1.000000   
1    7.818315e-01  0.623490  9.749279e-01 -0.222521  4.338837e-01 -0.900969   
2    9.749279e-01 -0.222521 -4.338837e-01 -0.900969 -7.818315e-01  0.623490   
3    4.338837e-01 -0.900969 -7.818315e-01  0.623490  9.749279e-01 -0.222521   
4   -4.338837e-01 -0.900969  7.818315e-01  0.623490 -9.749279e-01 -0.222521   
..            ...       ...           ...       ...           ...       ...   
664 -7.818315e-01  0.623490 -9.749279e-01 -0.222521 -4.338837e-01 -0.900969   
665 -1.952007e-15  1.000000 -3.904014e-15  1.000000 -2.332297e-13  1.000000   
666  7.818315e-01  0.623490  9.749279e-01 -0.222521  4.338837e-01 -0.900969   
667  9.749279e-01 -0.222521 -4.338837e-01 -0.900969 -7.818315e-01  0.623490   
668  4.338837e-01 -0.900969 -7.818315e-01  0.623490  9.749279e-01 -0.222521   

           sin7_4    cos7_4        sin7_5    cos7_5

In [17]:
# データを結合
df_tbl = pd.concat([dataX,
                    exog],
                   axis=1)

# 欠測値削除
df_tbl = df_tbl.dropna()

df_tbl.head()

,cdr_date,call_num,acc_get_cnt,acc_get_cnt_lag1,acc_get_cnt_lag7,acc_get_cnt_window7,acc_get_cnt_expanding,cm_flg,cm_cumsum,dow,...,sin7_3,cos7_3,sin7_4,cos7_4,sin7_5,cos7_5,sin7_6,cos7_6,sin7_7,cos7_7
7,2018-06-09,-0.801105,-0.297227,-0.569686,-0.743068,-0.035384,-0.035384,0,8,6,...,-7.347881e-16,1.000000,-9.797174e-16,1.000000,-1.224647e-15,1.000000,-1.469576e-15,1.000000,-1.714506e-15,1.0
8,2018-06-10,0.000000,-0.421072,-0.297227,-0.049538,0.028307,-0.068115,0,8,7,...,4.338837e-01,-0.900969,-4.338837e-01,-0.900969,-9.749279e-01,-0.222521,-7.818315e-01,0.623490,-1.959435e-15,1.0
9,2018-06-11,1.468692,0.978373,-0.421072,0.854528,-0.024769,-0.107332,0,8,1,...,-7.818315e-01,0.623490,7.818315e-01,0.623490,4.338837e-01,-0.900969,-9.749279e-01,-0.222521,-2.204364e-15,1.0
10,2018-06-12,-0.567449,-0.185767,0.978373,-0.210536,-0.007077,0.001238,0,8,2,...,9.749279e-01,-0.222521,-9.749279e-01,-0.222521,7.818315e-01,0.623490,-4.338837e-01,-0.900969,-2.449294e-15,1.0
11,2018-06-13,0.250345,0.359150,-0.185767,0.136229,-0.003538,-0.015762,0,8,3,...,-9.749279e-01,-0.222521,9.749279e-01,-0.222521,-7.818315e-01,0.623490,4.338837e-01,-0.900969,4.411204e-15,1.0


In [18]:
#連番の特徴量
df_tbl['t'] = pd.RangeIndex(start=0, stop=len(df_tbl))
df_tbl.tail()

,cdr_date,call_num,acc_get_cnt,acc_get_cnt_lag1,acc_get_cnt_lag7,acc_get_cnt_window7,acc_get_cnt_expanding,cm_flg,cm_cumsum,dow,...,cos7_3,sin7_4,cos7_4,sin7_5,cos7_5,sin7_6,cos7_6,sin7_7,cos7_7,t
664,2020-03-27,0.108483,0.185767,-0.099076,-0.718299,-0.024769,0.000951,1,203,5,...,-0.900969,4.338837e-01,-0.900969,9.749279e-01,-0.222521,7.818315e-01,0.623490,2.352708e-13,1.0,657
665,2020-03-28,-1.059795,-0.755453,0.185767,-0.359150,0.104383,0.001229,1,204,6,...,1.000000,-7.808028e-15,1.000000,-9.760036e-15,1.000000,-4.664594e-13,1.000000,-1.366405e-14,1.0,658
666,2020-03-29,0.000000,-0.421072,-0.755453,0.074307,0.047769,0.000093,1,205,7,...,-0.900969,-4.338837e-01,-0.900969,-9.749279e-01,-0.222521,-7.818315e-01,0.623490,-2.625989e-13,1.0,659
667,2020-03-30,1.118208,0.334381,-0.421072,0.817375,-0.023000,-0.000538,1,206,1,...,0.623490,7.818315e-01,0.623490,4.338837e-01,-0.900969,-9.749279e-01,-0.222521,-5.115338e-13,1.0,660
668,2020-03-31,-0.275380,0.953604,0.334381,0.173383,-0.091999,-0.000037,1,207,2,...,-0.222521,-9.749279e-01,-0.222521,7.818315e-01,0.623490,-4.338837e-01,-0.900969,1.490260e-13,1.0,661


In [19]:
df_tbl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 662 entries, 7 to 668
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   cdr_date                 662 non-null    datetime64[ns]
 1   call_num                 662 non-null    float64       
 2   acc_get_cnt              662 non-null    float64       
 3   acc_get_cnt_lag1         662 non-null    float64       
 4   acc_get_cnt_lag7         662 non-null    float64       
 5   acc_get_cnt_window7      662 non-null    float64       
 6   acc_get_cnt_expanding    662 non-null    float64       
 7   cm_flg                   662 non-null    int64         
 8   cm_cumsum                662 non-null    int64         
 9   dow                      662 non-null    int64         
 10  woy                      662 non-null    int64         
 11  wom                      662 non-null    int64         
 12  doy                      662 non-null    

# CSV出力

In [20]:
df_tbl.to_csv('dataset.csv', index=False)